# Segmenting and Clustering Neighborhoods in Toronto

##### Index of the notebook.
1. _Information retrival from wikipedia and storing into database,_
2. _Add neighbourhood latitude and longitude to the database,_
3. _Explore and cluster the neighborhoods in Toronto._

#### 1. Information retrival from Wikipedia and storing into database

In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Retrive the HTML code and create a BeautifulSoup object.
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = str(req.get(wiki_url).text)
soup=BeautifulSoup(wiki_page,'html.parser')

# Create a list with the informations contained in the table.
tag=soup.table
text=tag.get_text()
tmp_list=text.split('\n')
tmp_list2=tmp_list[1:-1]
new_list=[]
#print(tmp_list2) # uncomment to understand the for-cycle.

for i in range(0,len(tmp_list2),5):
    new_list.append([tmp_list2[i+1],tmp_list2[i+2],tmp_list2[i+3]])


# Create the database.
df_tor=pd.DataFrame(new_list[1:])
df_tor.columns=new_list[0]
df_tor.drop(df_tor[df_tor.Borough == 'Not assigned'].index, inplace=True) # Drop row with 'Borough' == 'Not assigned'.
df_tor.loc[df_tor['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df_tor.loc[df_tor['Neighbourhood'] == 'Not assigned', 'Borough']       # Replace when 'Neighbourhood' == 'Not assigne' with the 'Borough' name.        
df_tor.reset_index(drop=True,inplace=True) # Reset index to 0 after dropping row.

df_tor.head(20) #uncomment to see the first 20 row of the database

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


The above code uses BeautifulSoup functions in oder to get the text contained between the tags `<table>...</table>` 
used in the Wikipedia page to build a table. See comments in the code to understand the various instructions. The database assumes that, if not otherwise specified, the 'Borough' coincides with the 'Neighbourhood'.

In [2]:
df_tor.shape

(212, 3)

#### 2. Add neighbourhood latitude and longitude to the database

In [3]:
url_coord = 'http://cocl.us/Geospatial_data'
df_tor2 = pd.merge(left=df_tor,right=pd.read_csv(url_coord), how='left', left_on='Postcode', right_on='Postal Code')
df_tor2.drop('Postal Code',axis=1,inplace=True)
df_tor2.rename(columns={'Postcode':'Postal Code'},inplace=True)

The above code add latitude and longitude for each postal code by merging two databases. This is done since the geocode routine (install geocoder first)

```python
import geocoder 
lat_lng_coords = None
while(lat_lng_coords == None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng  
print(lat_lng_coords)
```

does not work, as anticipated in the assignment instructions.If one wants to obtain latitude and longitude for a given address (i.e. not using a postal code) the ```geopy``` library works very well.

In [4]:
df_tor2.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


#### 3. Explore and cluster the neighborhoods in Toronto.

In [5]:
# Packages installation, uncomment if needed

!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
print('\nDone.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

In [28]:
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 
import folium as fo
import datetime
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries loaded.')

Libraries loaded.


In [7]:
# Geo-query. (Do not work for postal code)

query = 'Toronto,Ontario'
geolocator = Nominatim()
location = geolocator.geocode(query)
lat_T = location.latitude
lon_T = location.longitude
print(lat_T,lon_T)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


43.653963 -79.387207


In [8]:
Toronto_map = fo.Map(location=[lat_T, lon_T], zoom_start=12)

Toronto_map

Let us now select only the ```Borough``` containing the word ```Toronto``` to reduce our dataset (i.e. we are considering mainly the historic part, a.k.a. the old Toronto, see https://en.wikipedia.org/wiki/Old_Toronto).

In [9]:
df_rest = df_tor2[df_tor2['Borough'].str.contains('Toronto')]

Now let us explore this dataset. It is important to check that there are no repetitions. Indeed, in this case all the ```groupby``` operations we will do to our pandas dataframe will alter the shape of our dataframes, generating errors.

In [10]:
print('Number of elements of df_rest[\'Neighbourhood\']:\t\t', len(df_rest['Neighbourhood']) ,\
      '\nNumber of unique elements of df_rest[\'Neighbourhood\']:\t', len(df_rest['Neighbourhood'].unique() ) )

Number of elements of df_rest['Neighbourhood']:		 74 
Number of unique elements of df_rest['Neighbourhood']:	 73


This means that there are two elements with the same Neighborhood name. Let us find out them.

In [11]:
from collections import Counter
repe=[item for item, count in Counter(df_rest['Neighbourhood']).items() if count > 1]
sel = df_rest['Neighbourhood']==repe[0]
df_double=df_rest[sel]
df_double

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
191,M4X,Downtown Toronto,St. James Town,43.667967,-79.367675


Hence ```St. James Town``` is preset two times. Let us print on the Toronto map these two points.

In [12]:
colors=['red','blue']
i=0

for lat, lng, bor, neigh,p_code in zip(df_double['Latitude'], df_double['Longitude'], df_double['Borough'], df_double['Neighbourhood'],df_double['Postal Code']):
    label = 'Postal Code: {}, Borough: {}, Neighborhoud: {}.'.format(p_code,bor,neigh)
    label = fo.Popup(label, parse_html=True)
    fo.CircleMarker([lat, lng],
                    radius=5,
                    popup=label,
                    color=colors[i],
                    fill=True,
                    fill_color='white',
                    fill_opacity=0.5,
                    parse_html=False
                   ).add_to(Toronto_map) 
    i=i+1
    
Toronto_map

Now, according to wikipedia (I am not from Toronto and not even Canada) the red dot is located in the Neighborhood called 'Old Town' (please see https://en.wikipedia.org/wiki/Old_Town,_Toronto), while the blue one is located in the Neighborhood called 'St. James Town' (please see https://en.wikipedia.org/wiki/St._James_Town). Hence we will change the name of the Neighborhood accordingly.

Note that _'Old Town'_ is missing in the list of postal code given for the project (see https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) and _'St. James Town'_ is present two times.

In [13]:
df_rest.loc[(df_rest['Neighbourhood']=='St. James Town')&(df_rest['Postal Code']=='M5C'),'Neighbourhood']='Old Town'

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Let us now check if we solve the issue of two Neighborhood having the same name.

In [14]:
print('Number of elements of df_rest[\'Neighbourhood\']:\t\t', len(df_rest['Neighbourhood']) ,\
      '\nNumber of unique elements of df_rest[\'Neighbourhood\']:\t', len(df_rest['Neighbourhood'].unique() ) )

Number of elements of df_rest['Neighbourhood']:		 74 
Number of unique elements of df_rest['Neighbourhood']:	 74


At this point we can finally plot the Toronto map with cicles on the center of each Neighborhood.

In [15]:
# Add markers to map.
for lat, lng, bor, neigh,p_code in zip(df_rest['Latitude'], df_rest['Longitude'], df_rest['Borough'], df_rest['Neighbourhood'],df_rest['Postal Code']):
    label = 'Postal Code: {}, Borough: {}, Neighborhoud: {}.'.format(p_code,bor,neigh)
    label = fo.Popup(label, parse_html=True)
    fo.CircleMarker([lat, lng],
                    radius=5,
                    popup=label,
                    color='black',
                    fill=True,
                    fill_color='coral',
                    fill_opacity=0.5,
                    parse_html=False
                   ).add_to(Toronto_map)  
    
Toronto_map

In [16]:
df_rest.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
27,M5C,Downtown Toronto,Old Town,43.651494,-79.375418


__Note that the function below would return an error message if you already finish the number of free query for 'Foursquare'. This is because it would not be able to correctly classify all the information contained in the error string (error 429).__

In [17]:
# Foursquare setting

CLIENT_ID='3FTNYFFWXYMEPZDEKDQE0O03VSBTSJK4SBRPPHRQH4GXLIYZ'
CLIENT_SECRET='3YRBYJH3ZOXHAV25ZCDCBJHCKRIG5DYQFXTTRJHZT304KALE'
VERSION=datetime.date.today().strftime("%Y%m%d")
LIMIT=100

# Retriving function

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)         
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)            
        results = req.get(url).json()["response"]['groups'][0]['items']   
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

# Retriving informations about restourant from Foursquare

df_info=getNearbyVenues(names=df_rest['Neighbourhood'],latitudes=df_rest['Latitude'],longitudes=df_rest['Longitude'])

Let us study how ```df_info``` look like.

In [18]:
print(df_info.shape)
df_info.head()

(3308, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Now we check if the column name ```Neighborhood``` is present among in venues category column (retrieved from Fourtsquare).

#### P.A. 
__Neighborhood and Neighbourhood (note the presence of 'u') has the same meaning but the first is in American English while the second is in British English. The database we obtain from Wikipedia uses the British version, while the database we obtain from the Foursquare uses the American one. Thus, likely they do not coincides. However in order to use a the built in function for data manipulation in Foursquare, the easiest solution is to change the name in Wikipedia database. This generate s the issue that we are going to solve below. Note that a trivial solution could be to change the names of the labels in the function ```getNearbyVenues()```.__

__By the way, I think it is a good habit to do this check (and eventually solve an issue like that) with real world database. Indeed you never know if some name of a database column coincides with a possible categorical variable.__

In [19]:
check_keyword = 'Neighborhood'
N=df_info['Venue Category'].str.contains('Neighborhood').sum()
print('Number of times \'Neighborhood\' appears in \'Venue Category\':',N)

Number of times 'Neighborhood' appears in 'Venue Category': 8


We have to take into account this fact later. Indeed when we transform the ```Venue Categroy``` categorical variable into numerical (using the function ```pd.get_dummies()```) we would have a column named ```Neighborhood```. To avoid problems, we will change the name of this column into  ```Neigborhood Venue Cat.```.

In [20]:
df_venues = pd.get_dummies(df_info[['Venue Category']], prefix="", prefix_sep="")
df_venues.rename(columns={'Neighborhood':'Neighborhood Venue Cat.'},inplace=True)
print(df_venues.shape)
df_venues.head()

(3308, 238)


,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


At this point we can add the column ```Neighborhood``` of the pandas dataframe ```df_info```

In [21]:
df_venues['ZZZ_Neighborhood'] = df_info['Neighborhood'] ### We added 'ZZZ_' to be sure that this is the last column.
fixed_columns = [df_venues.columns[-1]] + list(df_venues.columns[:-1])
df_info2=df_venues[fixed_columns]
df_info2.rename(columns={'ZZZ_Neighborhood':'Neighborhood'},inplace=True)
print(df_info2.shape)
df_info2.head()

(3308, 239)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_info2_grouped = df_info2.groupby('Neighborhood').mean().reset_index()


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues=10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_info2_grouped['Neighborhood']

for ind in np.arange(df_info2_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_info2_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Cosmetics Shop,Gym,Hotel,Restaurant
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boat or Ferry
2,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Steakhouse,Farmers Market,Seafood Restaurant,Pub,Café,Bakery
3,Brockton,Coffee Shop,Performing Arts Venue,Breakfast Spot,Café,Gym,Climbing Gym,Stadium,Bar,Burrito Place,Italian Restaurant
4,Business reply mail Processing Centre969 Eastern,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop,Spa


At this point we can apply the K Mean algorithm to generate clusters (unsupervised leanrning).

In [23]:
kclusters = 10 #our choice
df_info2_cluster = df_info2_grouped.drop('Neighborhood',axis=1)
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(df_info2_cluster)
df_rest.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
df_clust = df_rest
df_clust['Cluster labels'] = kmeans.labels_
df_clust = df_clust.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
df_clust.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Pop these, since the values are in `kwargs` under different names
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,7,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Restaurant,Mexican Restaurant,Theater,Spa
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,2,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Restaurant,Mexican Restaurant,Theater,Spa
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Tea Room,Burger Joint,Bubble Tea Shop
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,7,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Tea Room,Burger Joint,Bubble Tea Shop
27,M5C,Downtown Toronto,Old Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Hotel,Cosmetics Shop,Park,Cocktail Bar,Clothing Store,Bakery,Gastropub


In [29]:
map_clusters = fo.Map(location=[lat_T, lon_T], zoom_start=12)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df_clust['Latitude'], df_clust['Longitude'], df_clust['Neighborhood'], df_clust['Cluster labels']):
    label = fo.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    fo.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

At this point if you want you can explore each cluster. For example the first cluster can be visualized in the following way. It is characterized (mainly!) by Coffee Shop and Cafe' as most common venue.

In [25]:
df_clust.loc[df_clust['Cluster labels']==0,:]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Tea Room,Burger Joint,Bubble Tea Shop
27,M5C,Downtown Toronto,Old Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Hotel,Cosmetics Shop,Park,Cocktail Bar,Clothing Store,Bakery,Gastropub
49,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568,0,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Cosmetics Shop,Gym,Hotel,Restaurant
85,M4L,East Toronto,The Beaches West,43.668999,-79.315572,0,Sandwich Place,Park,Liquor Store,Steakhouse,Sushi Restaurant,Food & Drink Shop,Burrito Place,Ice Cream Shop,Fish & Chips Shop,Pub
131,M5R,Central Toronto,The Annex,43.672710,-79.405678,0,Coffee Shop,Café,Sandwich Place,Pizza Place,Pharmacy,French Restaurant,Cosmetics Shop,Pub,Burger Joint,Metro Station
143,M5S,Downtown Toronto,Harbord,43.662696,-79.400049,0,Bookstore,Café,Restaurant,Bar,Coffee Shop,Bakery,Japanese Restaurant,Nightclub,French Restaurant,Sandwich Place
